## plot strike, bid/ask, call, put amount v.s. PERP and underlying
   * using different marker size to represent amount of bid/ask
   * using diffrernt color to mark bid and ask 
   * left y-axis for strike
   * right y-axis for underlying price
   * hover mouse to show option price
   * 5 expire x put/call x bid/ask = 20 plots => 20 html files

In [ ]:
from cdcqr.common.config import LOCAL_DATA_DIR
from cdcqr.data.dataloader import data_loader
from cdcqr.data.deribit.data_utils import DeribitUtils
import pandas as pd
from datetime import datetime, timedelta
import plotly.express as px
import os
from cdcqr.common.config import LOCAL_FIGURE_DIR
import matplotlib.pyplot as plt
import plotly.graph_objects as go

### setup

In [ ]:
date = datetime(2021,10,29).date()
exchange = 'deribit'
maturity_date = datetime(2022,6,24).date()
maturity_date = datetime(2021,12,31).date()

freq = '1Min'
coin = 'BTC'

### load - option quote data

In [ ]:
symbol = 'OPTIONS'
data_type = 'quotes'
date_ = date.strftime('%Y-%m-%d')
opt_quote = data_loader(exchange, date_, data_type, symbol)
display(opt_quote.head(1))

### load - option chain data

In [ ]:
start_date = date
end_date = date
file_name = 'optchain_{}_{}_{}_{}.pkl'.format(start_date.strftime('%Y%m%d'), end_date.strftime('%Y%m%d'), maturity_date.strftime('%Y%m%d'), freq)
optchain = pd.read_pickle(os.path.join(LOCAL_DATA_DIR, file_name))
display(optchain.head(1))

### load - PERP data

In [ ]:
symbol = '{}-PERPETUAL'.format(coin)
data_type = 'quotes'
PERP_quote = data_loader(exchange, date.strftime('%Y-%m-%d'), data_type, symbol)
display(PERP_quote.head(1))

### data processing - all

In [ ]:
uni_index = pd.date_range(start=date, end=date+timedelta(days=1), freq='1Min')[1:]

#### data processing - PERP quote

In [ ]:
PERP_quote['dt'] = pd.to_datetime(PERP_quote['timestamp'], unit='us')
PERP_quote['PERP_mid_price'] = (PERP_quote['ask_price']+PERP_quote['bid_price'])/2

In [ ]:
PERP_quote_1 = PERP_quote.set_index('dt')['PERP_mid_price']
PERP_quote_2 = PERP_quote_1[~PERP_quote_1.index.duplicated(keep='last')]
PERP_quote_3 = PERP_quote_2.reindex(uni_index, method='ffill')

In [ ]:
PERP_quote_3

#### data processing - opt chain

In [ ]:
optchain['mid_vol'] = 0.5*(optchain['aiv'] + optchain['biv'])

In [ ]:
optchain_reduced = optchain[['symbol','strike','underly','s','aiv','biv']]
optchain_reduced = optchain_reduced[optchain_reduced['symbol'].str.contains('{}-'.format(coin))]
optchain_reduced = optchain_reduced.pipe(DeribitUtils.parse_optSymbol_col)

In [ ]:
# 1. get underlying price
optchain_reduced_1 = optchain_reduced['s']
optchain_reduced_2 = optchain_reduced_1[~optchain_reduced_1.index.duplicated(keep='last')]
optchain_reduced_2 = optchain_reduced_2.sort_index()
optchain_reduced_3 = optchain_reduced_2.reindex(uni_index, method='ffill').to_frame()

In [ ]:
# 2. get ask iv per strike, per type
optchain_reduced_1a = optchain_reduced[['aiv', 'biv', 'strike', 'type']]

In [ ]:
optchain_reduced_1a = optchain_reduced_1a.reset_index().rename(columns={'tm':'index'})

In [ ]:
# remove duplicates in opt chain
optchain_reduced_1a = optchain_reduced_1a[~optchain_reduced_1a.set_index(['type','strike','index']).index.duplicated(keep='last')]

In [ ]:
optchain_reduced_3.columns = ['underlying_price']
df_underlying = optchain_reduced_3.join(PERP_quote_3).reset_index()

In [ ]:
display(df_underlying.head(2))

In [ ]:
optchain_reduced_1a.head(2)

#### data processing - option quote

In [ ]:
opt_quote_parsed = opt_quote.pipe(DeribitUtils.parse_optSymbol_col)

In [ ]:
opt_quote_parsed['timestamp_dt'] = pd.to_datetime(opt_quote_parsed['timestamp'], unit='us')

In [ ]:
# only look at BTC
opt_quote_parsed = opt_quote_parsed.query('instrument=="BTC"')

In [ ]:
opt_quote_parsed

In [ ]:
# select relevant expire dates
selected_expr_dates = DeribitUtils.get_important_option_expire_dates(pd.to_datetime(date))
display(maturity_date)
display(selected_expr_dates)

In [ ]:
opt_quote_parsed['exp_date'] = opt_quote_parsed['expire'].dt.date

In [ ]:
opt_quote_parsed_expire_i_c = opt_quote_parsed.query('exp_date==@maturity_date & type =="C"')
opt_quote_parsed_expire_i_p = opt_quote_parsed.query('exp_date==@maturity_date & type =="P"')

In [ ]:
list(sorted(opt_quote_parsed_expire_i_c['strike'].unique())) == sorted(opt_quote_parsed_expire_i_c['strike'].unique())
strike_list = list(sorted(opt_quote_parsed_expire_i_c['strike'].unique()))
len(strike_list)

In [ ]:
opt_quote_parsed_expire_i_c_reduced = opt_quote_parsed_expire_i_c.drop(labels=['exchange','symbol','timestamp','local_timestamp','type','expire','t2m'], axis=1, errors='ignore')
opt_quote_parsed_expire_i_p_reduced = opt_quote_parsed_expire_i_p.drop(labels=['exchange','symbol','timestamp','local_timestamp','type','expire','t2m'], axis=1, errors='ignore')

In [ ]:
strike2df = {}
for strike in strike_list:
    #print(strike)
    opt_quote_parsed_expire_i_c_reduced_k_1 = opt_quote_parsed_expire_i_c_reduced.query('strike==@strike').set_index('timestamp_dt')
    opt_quote_parsed_expire_i_c_reduced_k_1 = opt_quote_parsed_expire_i_c_reduced_k_1[~opt_quote_parsed_expire_i_c_reduced_k_1.index.duplicated(keep='last')]
    opt_quote_parsed_expire_i_c_reduced_k_1 = opt_quote_parsed_expire_i_c_reduced_k_1.reindex(uni_index, method='ffill')
    strike2df[strike] = opt_quote_parsed_expire_i_c_reduced_k_1
    #display(opt_quote_btc_expire_i_c_reduced_k_1_1min.head(2))
df_combined = pd.concat(strike2df.values()).reset_index()
df_combined['ask_amount'] = df_combined['ask_amount'].fillna(0)
df_combined['bid_amount'] = df_combined['bid_amount'].fillna(0)
df_combined_c = df_combined.copy()

In [ ]:
strike2df = {}
for strike in opt_quote_parsed_expire_i_p_reduced['strike'].unique():
    #print(strike)
    opt_quote_parsed_expire_i_p_reduced_k_1 = opt_quote_parsed_expire_i_p_reduced.query('strike==@strike').set_index('timestamp_dt')
    opt_quote_parsed_expire_i_p_reduced_k_1 = opt_quote_parsed_expire_i_p_reduced_k_1[~opt_quote_parsed_expire_i_p_reduced_k_1.index.duplicated(keep='last')]
    opt_quote_parsed_expire_i_p_reduced_k_1_1min = opt_quote_parsed_expire_i_p_reduced_k_1.reindex(uni_index, method='ffill')
    strike2df[strike] = opt_quote_parsed_expire_i_p_reduced_k_1_1min
    #display(opt_quote_btc_expire_i_p_reduced_k_1_1min.head(2))
df_combined = pd.concat(strike2df.values()).reset_index()
df_combined['ask_amount'] = df_combined['ask_amount'].fillna(0)
df_combined['bid_amount'] = df_combined['bid_amount'].fillna(0)
df_combined_p = df_combined.copy()

In [ ]:
df_combined_c['type'] = 'C'
df_combined_p['type'] = 'P'
df_combined = df_combined_c.append(df_combined_p)

In [ ]:
df_combined['strike_shifted'] = df_combined['strike'] + 100*((df_combined['type']=='C').astype(int)-0.5)*2

In [ ]:
df_combined.head()

#### data processing - all

In [ ]:
df_combined_ext = pd.merge(left=df_combined, right = optchain_reduced_1a, on=['index','strike','type'], how='left')

In [ ]:
df_combined_ext['mvol'] = 0.5*(df_combined_ext['aiv']+df_combined_ext['biv'])

In [ ]:
display(df_combined.shape)
display(df_combined_ext.shape)

In [ ]:
df_combined_ext['total_amount'] = df_combined_ext['bid_amount'] + df_combined_ext['ask_amount']

### make plot
   * ask amount with call and put scatter plot
   * bid amount with call and put scatter plot
   * ask+bid amount with call and put scatter plot

In [ ]:
from plotly.subplots import make_subplots
subfig = make_subplots(specs=[[{"secondary_y": True}]])

In [ ]:
from plotly.subplots import make_subplots
subfig = make_subplots(specs=[[{"secondary_y": True}]])
fig1 = px.scatter(df_combined_ext, x="index", y="strike_shifted", size="ask_amount",  hover_data=['strike','ask_price','ask_amount','type', 'biv'], color="biv", title='',width=1500, height=1200)
fig1b = px.scatter(df_combined_ext, x="index", y="strike_shifted", size="bid_amount",  hover_data=['strike','bid_price','bid_amount','type', 'biv'], color="biv", title='',width=1500, height=1200)
fig1c = px.scatter(df_combined_ext, x="index", y="strike_shifted", size="total_amount", hover_data=['strike','ask_price','ask_amount','bid_price','bid_amount','type', 'aiv','biv','mvol'], color="mvol", title='',width=1500, height=1200)

In [ ]:
fig2 = px.line(df_underlying, x='index', y=['underlying_price', 'PERP_mid_price'], width=1500, height=1200)

In [ ]:
#df_combined_ext_20211029 = df_combined_ext.copy()
#%store df_combined_ext_20211029

In [ ]:
fig3 = go.Figure(data=fig1.data + fig2.data,)
fig3.update_layout(
    autosize=False,
    width=1500,
    height=1200,
    title='BTC ask price and amount v.s. underlying & PERP')
fig3.show()
fig3.write_html(os.path.join(LOCAL_FIGURE_DIR,'option_book',date.strftime('%Y%m'), 
                             "{}_options_{}_{}_ask.html".format(coin, date.strftime('%Y%m%d'), maturity_date.strftime('%Y%m%d'))))

In [ ]:
fig3 = go.Figure(data=fig1b.data + fig2.data,)
fig3.update_layout(
    autosize=False,
    width=1500,
    height=1200,
    title='BTC bid price and amount v.s. underlying & PERP')
fig3.show()
fig3.write_html(os.path.join(LOCAL_FIGURE_DIR,'option_book', date.strftime('%Y%m'),
                              "{}_options_{}_{}_bid.html".format(coin, date.strftime('%Y%m%d'), maturity_date.strftime('%Y%m%d'))))

In [ ]:
fig3 = go.Figure(data=fig1c.data + fig2.data,)
fig3.update_layout(
    autosize=False,
    width=1500,
    height=1200,
    title='BTC price and total amount v.s. underlying & PERP')
#fig.show()
fig3.write_html(os.path.join(LOCAL_FIGURE_DIR, 'option_book', date.strftime('%Y%m'),
                              "{}_options_{}_{}_all_quotes.html".format(coin, date.strftime('%Y%m%d'), maturity_date.strftime('%Y%m%d'))))

### plot for ATM options

In [ ]:
display(df_combined_ext.head())

In [ ]:
df_combined_ext_atm = df_combined_ext.query('strike==70000')
df_combined_ext_atm.head(2)

In [ ]:
df_combined_ext_p = df_combined_ext_atm.query('type=="P"')
df_combined_ext_c = df_combined_ext_atm.query('type=="C"')
df_combined_ext_p['type'] = 'P'
df_combined_ext_c['type'] = 'C'

In [ ]:
df_combined_ext_p_a = df_combined_ext_p[['index','ask_price', 'ask_amount', 'type', 'aiv']].rename(columns={'ask_price':'price', 'ask_amount':'amount', 'aiv':'iv'})
df_combined_ext_p_a['side'] = 'ask'
df_combined_ext_p_b = df_combined_ext_p[['index','bid_price', 'bid_amount', 'type', 'biv']].rename(columns={'bid_price':'price', 'bid_amount':'amount', 'biv':'iv'})
df_combined_ext_p_b['side'] = 'bid'

df_combined_ext_c_a = df_combined_ext_c[['index','ask_price', 'ask_amount', 'type', 'aiv']].rename(columns={'ask_price':'price', 'ask_amount':'amount', 'aiv':'iv'})
df_combined_ext_c_a['side'] = 'ask'
df_combined_ext_c_b = df_combined_ext_c[['index','bid_price', 'bid_amount', 'type', 'biv']].rename(columns={'bid_price':'price', 'bid_amount':'amount', 'biv':'iv'})
df_combined_ext_c_b['side'] = 'bid'

In [ ]:
df_combined_ext_transformed = df_combined_ext_p_a.append(df_combined_ext_p_b).append(df_combined_ext_c_a).append(df_combined_ext_c_b)
df_combined_ext_transformed.groupby(['side','type']).count()

In [ ]:
df_combined_ext_transformed.head()

In [ ]:
df_combined_ext_transformed['color'] = df_combined_ext_transformed['type']+ df_combined_ext_transformed['side']

In [ ]:
df_combined_ext_transformed.head(2)

In [ ]:
subfig = make_subplots(specs=[[{"secondary_y": True}]])
strike = 70000
fig1 = px.scatter(df_combined_ext_transformed, x="index", y="price", size="amount",  hover_data=['price','amount','type','side','iv'],symbol='type', color=df_combined_ext_transformed.color, title='',width=1500, height=1200)
fig1.show()
#fig1.update_yaxes(range=[60000, 80000])

In [ ]:
fig2 = px.line(df_underlying, x="index", y=['underlying_price','PERP_mid_price'], width=1500, height=1200, )
fig2.update_traces(yaxis="y2")
#fig2.update_yaxes(range=[60000, 80000])

In [ ]:
subfig.add_traces(fig1.data + fig2.data)
subfig.update_layout(
    autosize=False,
    width=1500,
    height=600,
    title='{} price and total amount v.s. underlying & PERP, date:{}, k:{}, expire:{} '.format(coin, date, strike, maturity_date),)

In [ ]:
df_combined_ext_transformed

In [ ]:
subfig.write_html(os.path.join(LOCAL_FIGURE_DIR, 'option_book', date.strftime('%Y%m'), "{}_option_{}_{}_{}.html".format(coin, date.strftime('%Y%m%d'), maturity_date.strftime('%Y%m%d'), strike)))